# sentinelRequest

sentinelRequest can be used to colocate a geodataframe (ie areas, trajectories, buoys, etc ...) with sentinel (1, but also 2 , 3 : all known by scihub)

## Install



```
% conda install  lxml numpy geopandas shapely requests fiona
% pip install git+https://gitlab.ifremer.fr/sarwing/sentinelrequest.git
```



## CLI usage

In [1]:
!sentinelRequest --help

Traceback (most recent call last):
  File "/home/losafe/users/oarcher/conda_sentinelRequest/bin/sentinelRequest", line 7, in <module>
    exec(compile(f.read(), __file__, 'exec'))
  File "/home1/datahome/oarcher/sarwing_project/sentinelrequest/bin/sentinelRequest", line 9, in <module>
    from sentinelRequest import scihubQuery
  File "/home1/datahome/oarcher/sarwing_project/sentinelrequest/sentinelRequest/__init__.py", line 499
    if not gdf_slice.empty: 
    ^
IndentationError: expected an indented block


### "One shot" from command line:

`
% sentinelRequest --user=xxxx --password=xxxxx --date='2018-09-23 00:00' --date='2018-09-23 12:00' --filename='S1?_?W_GRD*.SAFE' --cachedir=/home1/scratch/oarcher/scihub_cache/ --wkt='POLYGON ((-10 75, -10 86, 12 86, 12 84, -10 75))'
`

```
INFO:sentinelRequest:from 2018-09-23 00:00:00 to 2018-09-23 12:00:00 : 11 SAFES
INFO:sentinelRequest:Total : 11 SAFES
filename
S1B_EW_GRDM_1SDH_20180923T071854_20180923T071954_012839_017B47_17F2.SAFE
S1B_EW_GRDM_1SDH_20180923T071954_20180923T072054_012839_017B47_1E6F.SAFE
S1B_EW_GRDM_1SDH_20180923T072054_20180923T072154_012839_017B47_CD41.SAFE
S1B_EW_GRDM_1SDH_20180923T072154_20180923T072254_012839_017B47_3682.SAFE
S1A_EW_GRDM_1SDH_20180923T081003_20180923T081107_023823_02997B_049A.SAFE
S1A_EW_GRDM_1SDH_20180923T081107_20180923T081207_023823_02997B_6EA6.SAFE
S1B_EW_GRDM_1SDH_20180923T085656_20180923T085756_012840_017B4E_B07B.SAFE
S1B_EW_GRDM_1SDH_20180923T085756_20180923T085856_012840_017B4E_6CAD.SAFE
S1B_EW_GRDM_1SDH_20180923T085856_20180923T085956_012840_017B4E_1CCD.SAFE
S1B_EW_GRDM_1SDH_20180923T103504_20180923T103604_012841_017B54_DBBC.SAFE
S1B_EW_GRDM_1SDH_20180923T103604_20180923T103704_012841_017B54_B267.SAFE
```

### From csv file

`
% cat test.csv
`
```
index;startdate;stopdate;geometry
area1;2018-10-02 00:00;2018-10-02 21:00;POLYGON ((-12 35, -5 35, -5 45, -12 45, -12 35))
area2;2018-10-13 06:00;2018-10-13 21:00;POLYGON ((-10 32, -3 32, -3 42, -10 42, -10 32))
area3;2018-10-13 00:00;2018-10-13 18:00;POLYGON ((12 35, 5 35, 5 45, 12 45, 12 35))
```

`
% sentinelRequest --user=xxxx --password=xxxx --infile=test.csv --filename='S1?_?W_GRD*.SAFE' --cachedir=/home1/scratch/oarcher/scihub_cache/ --cols=index,filename
`

```
INFO:sentinelRequest:req 1/2 from 2018-10-02 00:00:00 to 2018-10-02 21:00:00 : 9/21 SAFES
INFO:sentinelRequest:req 2/2 from 2018-10-13 00:00:00 to 2018-10-13 21:00:00 : 30/35 SAFES
INFO:sentinelRequest:Total : 39 SAFES
index;filename
area1;S1A_IW_GRDH_1SDV_20181002T061827_20181002T061852_023953_029DA0_C61E.SAFE
area1;S1B_IW_GRDH_1SDV_20181002T181105_20181002T181130_012977_017F7D_FE88.SAFE
area1;S1B_IW_GRDH_1SDV_20181002T181130_20181002T181155_012977_017F7D_93FF.SAFE
area1;S1B_IW_GRDH_1SDV_20181002T181155_20181002T181222_012977_017F7D_CD9A.SAFE
area3;S1A_IW_GRDH_1SDV_20181013T053545_20181013T053610_024113_02A2DB_D121.SAFE
area3;S1A_IW_GRDH_1SDV_20181013T053815_20181013T053840_024113_02A2DB_7D53.SAFE
area2;S1B_IW_GRDTH_1SDV_20181013T062502_20181013T062527_013130_018428_1E77.SAFE
area2;S1B_IW_GRDH_1SDV_20181013T062527_20181013T062552_013130_018428_82AB.SAFE
area2;S1B_IW_GRDH_1SDV_20181013T062642_20181013T062707_013130_018428_AB0E.SAFE
area2;S1B_IW_GRDH_1SDV_20181013T062707_20181013T062732_013130_018428_8210.SAFE
```

If `--date` is specified 2 times with `--infile`, it will superseeds ones founds in infile :

`
sentinelRequest --user oarcher --password nliqt6u3 --infile=test.csv --date=last-monday-7days --date=now --filename='S1?_?W_GRD*.SAFE' --cachedir=/home1/scratch/oarcher/scihub_cache/ --cols=index,filename
`



## API usage

In [2]:
%matplotlib inline
import geopandas as gpd
from sentinelRequest import scihubQuery
import datetime
import matplotlib.pyplot as plt
import shapely.wkt as wkt

# get your own credential from  https://scihub.copernicus.eu/dhus
import pickle
user,password = pickle.load(open("credential.pkl","rb"))

IndentationError: expected an indented block (__init__.py, line 499)

In [ ]:
help(scihubQuery)

### Example 1 

As an example, tree areas are defined:
* Ireland, that space overlap with Gascogne, but not overlap it in time.
* Gascogne and Portugal, that overlap in time and space

In [ ]:
gdf = gpd.GeoDataFrame({
        "beginposition" : [ datetime.datetime(2018,10,2,0), datetime.datetime(2018,10,13,6) ,  datetime.datetime(2018,10,13,0) ],
        "endposition"   : [ datetime.datetime(2018,10,2,21) , datetime.datetime(2018,10,13,21) ,datetime.datetime(2018,10,13,18) ],
        "geometry"      : [ wkt.loads("POINT (-7 53)").buffer(4), wkt.loads("POINT (-5 45)").buffer(5),  wkt.loads("POLYGON ((-12 35, -5 35, -5 45, -12 45, -12 35))")]    
    },index=["Irland","Gascogne","Portugal"])
gdf

In [ ]:
fig = plt.figure(figsize=(10,7))
safes = scihubQuery(
        gdf=gdf,
        min_sea_percent=20, 
        user=user,
        password=password,
        filename='S1?_?W_GRD*.SAFE',
        cachedir='/home1/scratch/oarcher/scihub_cache',
        fig=fig)


User requested area are in green. They can be more complex shape than allowed by scihub, because shapes are simplified for the request (in red ). Once the request is done, a colocation is done with the originals shapes, to drop extraneaous safes (in yellow).

Gascogne and Portugal areas are merged into a simplier big shape, because they are adjacent in time and space. Ireland is adjacent in space, but not in time (see example 2 for more infos).

Green safes are safes over land (min_sea_percent).


The result is a geodataframe with most information from scihub. Only colocated (ie blue) are returned:

In [ ]:
safes.iloc[0]

Most fields are converted from str to python type (geometry, datetime, int ...)

In [ ]:
safes['footprint'].plot()

In [ ]:
print('safe was ingested %s after aquisition' % (safes.iloc[0]['ingestiondate']-safes.iloc[0]['endposition']))

Index from original request are preserved, so it's easy to know the area that belong to a safe. (See end of example 2 for advanced index handling).

In [ ]:
safes.loc['Portugal']

Some safes are in both area (Portugal and Gascogne), because they overlap in space and time.

In [ ]:
safes[safes.duplicated(['filename'],keep=False)][['filename']]

### Example 2
Cyclone track colocalization

In [ ]:
import pandas as pd
#ibtracs=gpd.read_file('tmp/IBTrACS.NA.list.v04r00.points.shp')
#gdf_track=ibtracs[ibtracs['SID'] == '2019235N10324']
#gdf_track=gdf_track[['ISO_TIME','USA_WIND','geometry']]
#gdf_track['ISO_TIME']=pd.to_datetime(gdf_track['ISO_TIME'],format="%Y-%m-%d %H:%M:%S")
#gdf_track.reset_index(inplace = True,drop=True) 
#gdf_track.to_file("track.gpkg", driver="GPKG")

gdf_track = gpd.read_file('track.gpkg')
gdf_track['ISO_TIME']=pd.to_datetime(gdf_track['ISO_TIME'],format="%Y-%m-%d %H:%M:%S")
gdf_track

In [ ]:
fig = plt.figure(figsize=(10,7))
safes = scihubQuery(
        gdf=gdf_track,
        date='ISO_TIME', # no startdate/stopdate, but a date ans a dtime
        dtime=datetime.timedelta(hours=1.5),
        user=user,
        password=password,
        datatake=1,   # take adjacents safes, up to one.
        filename='S1?_?W_GRD*.SAFE',
        cachedir='/home1/scratch/oarcher/scihub_cache',
        fig=fig)

#### datatake

Here, `datatake=1` is specified to retrieve adjacents safes from colocated ones (in cyan). When specified, the result contain a `datatake_index` column. 0 means the colocated one, and other values are the range of the adjacent safe (up to -n..n with `datatake=n`)

Positive `datatake_index` are for safes *after* the colocated one, and negative index are fo safes *before* the colocated one.

In [ ]:
safes[['filename','datatake_index']]

#### Time slicing with timedelta_slice

One can see on previous figure that 3 requests are done (ie 3 red area). gdf rows are grouped to reduce the amount of scihub requests with the `timedelta_slice` parameter (default to `datetime.timedelta(weeks=1)` )

If we reduce `timedelta_slice`, we can see that more scihub request are done, with less uncolocated safes (ie yellow). (be warned with a big `timedelta_slice` : this can produce scihub timeouts).

(with `timedelta_slice=None`, this feature is *disabled* : a scihub request is done for *every* geometry).

In [ ]:
# same request as above, but with reduced timedelta_slice
fig = plt.figure(figsize=(10,7))
safes = scihubQuery(
        gdf=gdf_track,
        date='ISO_TIME',
        dtime=datetime.timedelta(hours=1.5),
        user=user,
        password=password,
        timedelta_slice=datetime.timedelta(days=1),
        datatake=1,
        filename='S1?_?W_GRD*.SAFE',
        cachedir='/home1/scratch/oarcher/scihub_cache',
        fig=fig)

#### Merging source and result with shared index

As seen before, the result (safes) share the same index as the source. So we can merge the two geodataframe, to associate a wind speed from the cyclone track with the safe, and compute distance from the eye to the safe.

In [ ]:
# here, we merge the result with the source request, to associate wind speed to each safe.
merged=safes[['filename','datatake_index','footprint']].merge(
    gdf_track[['USA_WIND','geometry']],left_index=True,right_index=True)
merged['eye_dist'] = merged.set_geometry('geometry').distance(merged.set_geometry('footprint').exterior)
# negative dist if safe contains eye
merged['eye_dist']=merged['eye_dist']*(((~merged.set_geometry('footprint').contains(merged.set_geometry('geometry'))+1)*2)-3)
merged[['filename','datatake_index','USA_WIND','eye_dist']]


## Annexes

### Antimeridian handling: small geometry vs large one


Given 2 points on the earth, there is two possible paths: one short, and one long that wrap around the earth.

By default, geometry are the smallest ones. To preserve a large geometry,  GeometryCollection must be used.


In [ ]:
from shapely.geometry import GeometryCollection

# the polygon is more than 180 deg wide : it will be wrapped, and will cross antimeridian
large_poly = wkt.loads("POLYGON ((-140 -14, 140 -14, 140 -20, -140 -20, -140 -14))")

gdf = gpd.GeoDataFrame({
        "beginposition" : [ datetime.datetime(2018,10,1)],
        "endposition"   : [ datetime.datetime(2018,10,31) ],
        "geometry"      : [ large_poly ]    
    },index=[0])

fig = plt.figure(figsize=(10,7))
safes = scihubQuery(
        gdf=gdf,
        user=user,
        password=password,
        filename='S1?_?W_GRD*.SAFE',
        cachedir='/home1/scratch/oarcher/scihub_cache',
        fig=fig)
plt.show()

# same polygon, but encapsulated in a GeometryCollection : it will not be wrapped
gdf = gpd.GeoDataFrame({
        "beginposition" : [ datetime.datetime(2018,10,1)],
        "endposition"   : [ datetime.datetime(2018,10,31) ],
        "geometry"      : [ GeometryCollection([large_poly]) ]    
    },index=[0])

fig = plt.figure(figsize=(10,7))
safes = scihubQuery(
        gdf=gdf,
        user=user,
        password=password,
        filename='S1?_?W_GRD*.SAFE',
        cachedir='/home1/scratch/oarcher/scihub_cache',
        fig=fig)
plt.show()

gdf